To do pages:
http://stefansavev.com/blog/text-classification-with-scikit-learn/
http://fastml.com/classifying-text-with-bag-of-words-a-tutorial/
http://www.sohu.com/a/130642494_355135?_f=v2-index-feeds
http://www.cnblogs.com/DjangoBlog/p/6202507.html
http://albertxiebnu.github.io/fasttext/

# AI100 Text classification introduction

Here are comparisions of different strategies for text classification task on [AI100](http://competition.ai100.com.cn/html/game_det.html?id=24&tab=1).

# Part 1: Bag of words

First we try traditional machine learning strategy, i.e., [bag of words (bow)](https://en.wikipedia.org/wiki/Bag-of-words_model) + different algorithms. 

For unstructured text data, we should extract numerical features vectors first. 

Here we use bag of words to represent text. The training data (corpus) are converted to a matrix, which each row represent document and columns are word vocabulary appeared in the whole corpus. And the element of the matrix can be word counts or tf-idf. Please refer to [here](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html) for more details.

Note that the above matrix is high dimensional sparse !

## Step1: reading input and text segmentation

In this section, we will import necessary packages first (such as [scikit-learn](http://scikit-learn.org/), [pandas](http://pandas.pydata.org/), [numpy](http://www.numpy.org/), [jieba](https://github.com/fxsjy/jieba)), load input data and perform Chinese text segmentation.

In [83]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import jieba as jb
import codecs
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
pd.set_option('display.max_colwidth', -1)

In [84]:
data_dir = 'materials' # input file directory
# read input
training_data = pd.read_csv(os.path.join(data_dir, 'training.csv'), names=['text_label', 'text_content'], encoding='utf8')

In [85]:
# size of training data
training_data.shape

(4774, 2)

In [ ]:
# take a glance 
training_data

In [ ]:
# compute the number of each category
pd.value_counts(training_data['text_label'].values)#.plot(kind='bar')
# or training_data.groupby(training_data['text_label']).count().plot(kind='bar')

In [86]:
# helper function: reading file
def read_file(file_path):
    f = codecs.open(file_path, encoding='utf-8')
    lines = []
    for line in f:
        line = line.rstrip('\n').rstrip('\r')
        lines.append(line)
    return lines

stopwordsCN = read_file(os.path.join(data_dir, 'stopWords_cn.txt'))

# helper function: text segmentation
def cut_content(each_row):
    return ' '.join([word for word in jb.lcut(each_row['text_content']) if word not in stopwordsCN])

In [87]:
# perform text segmentation
training_data['text_content_segmentation'] = training_data.apply(cut_content, axis=1)

In [ ]:
# look at the first ten rows results
training_data.head(10)['text_content_segmentation']

In [ ]:
new_words = read_file(os.path.join(data_dir, 'ai100_words.txt'))
for word in new_words:
    jb.add_word(word)

training_data['text_content_segmentation'] = training_data.apply(cut_content, axis=1)
#testing_data.head(10)
selected_index = range(1001, 1200)
print selected_index
training_data.iloc[selected_index]

In [88]:
from tgrocery import Grocery
# Create a grocery(don't forget to set a name)
grocery = Grocery('sample')
train_src = []
for index, row in training_data.iterrows():
    train_src.append((row['text_label'], row['text_content_segmentation']))
    
print 'len of train_src:{}'.format(len(train_src))


grocery.train(train_src)


def grocery_predict(each_row):
    return grocery.predict(each_row['text_content_segmentation'])

print 'predict'
training_data['predict_label'] = training_data.apply(grocery_predict, axis=1)

print 'result:'
print np.mean(training_data['predict_label'] == training_data['text_label'])

WindowsError: [Error 126] The specified module could not be found

## Step 2: extracting features

In this section, we will extract tf-idf numerical features using [CounterVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) and [TfidfTransformer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) classes.

In [ ]:
# word count feature
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data['text_content_segmentation'])
# play with X_train_counts: https://de.dariah.eu/tatom/working_with_text.html

In [ ]:
type(X_train_counts)

In [ ]:
X_train_counts.shape

In [ ]:
count_vect.vocabulary_#.get(u'企业')

In [ ]:
# tf-idf feature
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
type(X_train_tfidf)

In [ ]:
X_train_tfidf.shape

## Step 3: build classifier

In this section, we use linear [Support Vector Machine (SVM)](https://github.com/jakevdp/sklearn_pycon2015/blob/master/notebooks/03.1-Classification-SVMs.ipynb) method to build the model.

In [ ]:
# linear SVM
# http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html
svm_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5)

In [ ]:
# 10-fold cross-validation results
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
cv_scores = cross_val_score(svm_clf, X_train_tfidf, training_data['text_label'], cv=KFold(n_splits=10), n_jobs=-1)

In [ ]:
cv_scores

In [ ]:
np.mean(cv_scores)

## Step 4: pipeline

In this section, we build a [pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) for feature extracting and model building.

In [ ]:
text_clf = Pipeline([('word_counter', CountVectorizer()),
                    ('tfidf_computer', TfidfTransformer()),
                    ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5))])

In [ ]:
text_clf = text_clf.fit(training_data['text_content_segmentation'], training_data['text_label'])

In [ ]:
cv_scores = cross_val_score(text_clf, training_data['text_content_segmentation'], training_data['text_label'], cv=KFold(n_splits=10), n_jobs=-1)

In [ ]:
np.mean(cv_scores)

## Step 5: predicting

In this section, the above model is used to preform predicting.

In [ ]:
new_words = read_file(os.path.join(data_dir, 'ai100_words.txt'))
for word in new_words:
    jb.add_word(word)

In [ ]:
# read the test data
testing_data = pd.read_csv(os.path.join(data_dir, 'testing.csv'), names=['text_index', 'text_content'], encoding='utf-8')

In [ ]:
testing_data.shape

In [ ]:
testing_data['text_content_segmentation'] = testing_data.apply(cut_content, axis=1)

In [ ]:
#testing_data.head(10)
selected_index = range(81, 90)
print selected_index
testing_data.iloc[selected_index]

In [ ]:
predict_res = text_clf.predict(testing_data['text_content_segmentation'])

In [ ]:
predict_res

In [ ]:
# save to csv file
np.savetxt(os.path.join(data_dir, 'results.csv'), np.dstack((np.arange(1, predict_res.size+1), predict_res))[0],"%d,%d")
# submit accuracy: 0.836

## Step 6: debugging

Several directions can be done...

[1] model parameter tuning, riched feature (n-gram or word2vec)

[2] feature selection

[3] try XGBoost and liblinear

[4] model ensemble

[5] deal with imbalanced dataset

[6] better text segmentation vocabulary

### The following codes shows the solution of [1].

In [ ]:
text_clf = Pipeline([('word_counter', CountVectorizer()),
                    ('tfidf_computer', TfidfTransformer()),
                    ('clf', SGDClassifier(loss='hinge'))])
parameters = {
    'word_counter__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'word_counter__min_df': (1, 2, 3, 4, 5),
    
    'tfidf_computer__norm': ('l1', 'l2'),
    'tfidf_computer__use_idf': (True, False),
    'tfidf_computer__smooth_idf': (True, False),
    'tfidf_computer__sublinear_tf': (True, False)
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(training_data['text_content_segmentation'], training_data['text_label'])

print gs_clf.best_score_
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
# submit accuracy: 0.858

### We try [2] feature selection.

### We then [3] try XGBoost and liblinear.

[XGBoost](http://xgboost.readthedocs.io/en/latest/) is a very popular package in [Kaggle](https://www.kaggle.com/) competiton. Here, we have a look at the power of XGBoost.

In [ ]:
# references: 
# http://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/
# https://jessesw.com/XG-Boost/


### Now, we do [4] model ensemble.

In [ ]:
#http://sebastianraschka.com/Articles/2014_ensemble_classifier.html
#https://stats.stackexchange.com/questions/190151/ensembling-with-votingclassifier
#http://machinelearningmastery.com/ensemble-machine-learning-algorithms-python-scikit-learn/
#http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html

'''
from sklearn.ensemble import VotingClassifier

estimators = []
svm_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, n_iter=5)
estimators.append(('svm', svm_clf))

mnb = MultinomialNB(alpha=0.1)
estimators.append(('nb', mnb))

rf_clf = RandomForestClassifier(n_estimators=50)
estimators.append(('rf', rf_clf))

voting = VotingClassifier(estimators)
'''

### And we [5] deal with imbalanced dataset.

In [ ]:
# http://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
# http://blog.csdn.net/heyongluoyao8/article/details/49408131
# http://www.jianshu.com/p/3e8b9f2764c8
# https://github.com/ThoughtWorksInc/dataclouds/blob/master/source/_posts/%E4%B8%8D%E5%B9%B3%E8%A1%A1%E6%95%B0%E6%8D%AE%E5%A4%84%E7%90%86.md
    
sample_size = 200

def sample_data(each_group, sam_size):
    if len(each_group.index) > sam_size:
        return each_group.loc[np.random.choice(each_group.index, sam_size, False),:]
    else:
        return each_group
    
banlanced_data = training_data.groupby(training_data['text_label']).apply(lambda x: sample_data(x, sample_size))

In [ ]:
#pd.value_counts(training_data['text_label'].values)
pd.value_counts(banlanced_data['text_label'].values)

In [ ]:
text_clf = text_clf.fit(banlanced_data['text_content_segmentation'], banlanced_data['text_label'])

In [ ]:
cv_scores = cross_val_score(text_clf, banlanced_data['text_content_segmentation'], banlanced_data['text_label'], cv=KFold(n_splits=10), n_jobs=-1)

In [ ]:
np.mean(cv_scores)

# Part 2: LDA

In [ ]:
https://gist.github.com/aronwc/8248457
http://mccormickml.com/2016/03/25/lsa-for-text-classification-tutorial/
https://github.com/chrisjmccormick/LSA_Classification/blob/master/runClassification_LSA.py

# Part 3: Word2vec

[Word2vec](https://en.wikipedia.org/wiki/Word2vec) is a group of related models (continuous bag-of-words (CBOW) and skip-gram) that are used to produce word embeddings, which is a distributed representation of words. 

Question: 'man' - 'woman' = 'king' - ?

Here, we use [gensim](https://radimrehurek.com/gensim/) package to implement Word2vec.

## Solution 1: Word2vec + SVM

In [ ]:
http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/
%load materials/word2vec/word2vec+svm.py

## Solution 2: Doc2vec + SVM

In [ ]:
#http://www.shuang0420.com/2016/06/01/gensim-doc2vec%E5%AE%9E%E6%88%98/
#https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-IMDB.ipynb
#http://stackoverflow.com/questions/31321209/doc2vec-how-to-get-document-vectors
#https://datascience.stackexchange.com/questions/10216/doc2vec-how-to-label-the-paragraphs-gensim
#https://groups.google.com/forum/#!topic/word2vec-toolkit/HpVnMfeo5PM
#https://github.com/linanqiu/word2vec-sentiments/blob/master/word2vec-sentiment.ipynb
#http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/

## Solution 3: FastText

Maybe due to the lack of more training data, the accuracy of FastText is about 75%.

Step 1: prepare training data

In [ ]:
%load materials/Fasttext/AI100_step1_prepare_training_data.py

Step 2: training the model

In [ ]:
%load materials/Fasttext/AI100_step2_training.py

Step 3: predicting

In [ ]:
%load materials/Fasttext/AI100_step3_predicting.py

## Solution 4: Word2vec + CNN

To do...

# 